In [1]:
import numpy as np
from codebase.file_utils import save_obj, load_obj, make_folder, path_backslash
from pdb import set_trace


In [19]:
y_t = np.array([1,1,0,0,1,0])
theta_t=dict()
theta_t['alpha'] = np.array([0.3, 0.1, 0.4, 0.6, -0.8, 0.1])
theta_t['beta'] = np.ones((6,1))*0.3
z_t = np.array([[1]])


In [20]:
def get_pi_z(z, theta):
    exp_eta = np.exp(theta['alpha'] +  z @ theta['beta'].T)
    return exp_eta/(1+exp_eta)

pi_z = get_pi_z(z_t, theta_t)

In [45]:
def get_log_likelihood(z,y,theta):
    pi_z = get_pi_z(z, theta)
    s1 = np.sum((y*np.log(pi_z))+((1.-y)*(1.-np.log(pi_z))))
    s2 = -.5 * np.sum(z**2)
    return s1+s2

def get_neg_log_likelihood(z,y,theta):
    return - get_log_likelihood(z,y,theta)

lglk = get_neg_log_likelihood(z_t, y_t, theta_t)
lglk

-1.8327749889515537

In [46]:
def get_grad_pi_z(z, theta):
    exp_eta = np.exp(theta['alpha'] +  z @ theta['beta'].T)
    return (exp_eta @  theta['beta'])/(1+exp_eta)**2

get_grad_pi_z(z_t, theta_t)

array([[0.37237122, 0.47763013, 0.32652126, 0.2477845 , 1.14907574,
        0.47763013]])

In [47]:
def get_hessian(z, y, theta):
    pi_z = get_pi_z(z, theta)
    grad_pi_z = get_grad_pi_z(z, theta)
    r1 =grad_pi_z**2
    r2 =pi_z*(1.-pi_z)
    return 1. + np.sum(r1/r2)

get_hessian(z_t, y_t, theta_t)

9.903271758875007

##

In [48]:
from scipy.optimize import brentq, minimize
def find_arg_max_loglikelihood(z, y, theta):
    

    return phi

In [49]:
from scipy.optimize import brentq, minimize
x0 = z_t.copy()*0.1
res = minimize(get_neg_log_likelihood, x0, args=(y_t, theta_t), method='BFGS')

In [50]:
res.x

array([0.10803739])

In [51]:
get_neg_log_likelihood(z_t, y_t, theta_t)

-1.8327749889515537

In [55]:
get_log_likelihood(z_t, y_t, theta_t)

1.8327749889515537

In [56]:
get_log_likelihood(res.x, y_t, theta_t)

2.2308799758946907

In [61]:
fisher_info_matrix = get_hessian(res.x, y_t, theta_t)
fisher_info_matrix


9.635156620750225

In [60]:
from scipy.stats import multivariate_normal

In [65]:
laplace_samples = multivariate_normal.rvs(mean = res.x, cov = fisher_info_matrix**(-1), size = 1000)

In [66]:
from codebase.plot import *

In [69]:
df = pd.DataFrame(laplace_samples, columns=['value'])

In [73]:
c = alt.Chart(df).transform_fold(
        ['value']
        ).transform_density(
            density='value',
        ).mark_area(opacity=0.5).encode(
            alt.X('value:Q', title=None),
            alt.Y('density:Q')
)
c

alt.Chart(...)